# Predicting Student Exam Scores Using a Deep Learning model
### (Not finished)

In [ ]:
import torch
import pandas as pd
import numpy as np
import kaggle as kg
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def read_data(file_path):
    data = pd.read_csv(file_path)
    return data

def preprocess_and_scaling(data):
    # Preprocess and scale the data
    # Placeholder for actual preprocessing and scaling logic
    return data

#Function for splitting data into training and validation sets
def split_data(data, val_size=0.2):
    train_data, val_data = train_test_split(data, test_size=val_size, random_state=42)
    return train_data, val_data
    

#Dataloader:
class StudentDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        features = sample.drop('exam_score').values.astype(np.float32)
        target = sample['exam_score'].astype(np.float32)
        return features, target
    
def create_dataloader(data, batch_size=128, shuffle=True):
    dataset = StudentDataset(data)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader




## Build model:

I decided to use a simple architecture for the model, a MLP with 4 layers.


In [ ]:
class NeuralNetworkModel(torch.nn.Module):
    def __init__(self, input_size):
        super(NeuralNetworkModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 64)
        self.fc2 = torch.nn.Linear(64, 128)
        self.fc3 = torch.nn.Linear(128, 64)
        self.fc4 = torch.nn.Linear(64, 32)
        self.fc5 = torch.nn.Linear(32, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    

### Define Function for training model using tqdm to keep track of progress

In [ ]:
# Train model, keep track of loss with tqdm progress bar.
#Save Validation and train losses for each epoch.
def train_model(model, dataloader_train, dataloader_val, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in tqdm(dataloader_train, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(dataloader_train.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in dataloader_val:
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item() * inputs.size(0)
        val_epoch_loss = val_loss / len(dataloader_val.dataset)
        print(f"Validation Loss: {val_epoch_loss:.4f}")
        model.train()


In [ ]:
#